# Project Setup

This notebook will guide you through setting up the project environment for using CrewAI. We will:

1. Install the required Python modules.
2. Set up a virtual environment.
3. Verify the installation.

In [ ]:
# Step 1: Create and activate a virtual environment
%python3 -m venv venv
%source venv/bin/activate

# Step 2: Install required Python modules
%pip install -r requirements.txt

# Step 3: Verify installation
%pip list

# Setting Up OpenAI API Key

To use CrewAI with the OpenAI API, you need to set your OpenAI API Key as an environment variable. This key is required for CrewAI to interact with OpenAI's services. Replace `your_openai_api_key` with your actual API key in the code block below.

In [1]:
# Step 4: Set OpenAI API Key from .env file
import os
from dotenv import load_dotenv
from IPython.display import Markdown
from crewai import LLM, Agent, Task, Crew

# Load environment variables from .env file
load_dotenv()

# Uncomment the code block below to use OpenAI with your API Key
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the .env file")

llm = LLM(
    model="gpt-4o",  # Specify the OpenAI model you want to use
    api_key=api_key
)

# Uncomment the code block below to use Ollama with your local model
# Make sure to have Ollama installed and running
# 
# llm = LLM(
#     model="ollama/llama3.2:1b",
#     base_url="http://localhost:11434"
# )



# Define Agents
This block defines multiple agents with specific roles, goals, and backstories. Each agent is configured to use the LLM defined earlier.

In [2]:
senior_technical_writer = Agent(
    role="Senior Technical Writer",
    
    goal="""Craft clear, engaging, and well-structured
            technical content based on research findings""",
    
    backstory="""You are an experienced technical writer
                with expertise in simplifying complex
                concepts, structuring content for readability,
                and ensuring accuracy in documentation.""",
                
    llm=llm,
                
    verbose=True
)

research_analyst = Agent(
    role="Senior Research Analyst",
    goal="""Find, analyze, and summarize information 
            from various sources to support technical 
            and business-related inquiries.""",
    backstory="""You are a skilled research analyst with expertise 
                in gathering accurate data, identifying key trends, 
                and presenting insights in a structured manner.""",
    llm=llm,
    verbose=True
)

code_reviewer = Agent(
    role="Senior Code Reviewer",
    goal="""Review code for bugs, inefficiencies, and 
            security vulnerabilities while ensuring adherence 
            to best coding practices.""",
    backstory="""You are a seasoned software engineer with years of 
                experience in writing, reviewing, and optimizing 
                production-level code in multiple programming languages.""",
    llm=llm,
    verbose=True
)

legal_reviewer = Agent(
    role="Legal Document Expert Reviewer",
    goal="""Review contracts and legal documents to 
            ensure compliance with applicable laws and 
            highlight potential risks.""",
    backstory="""You are a legal expert with deep knowledge 
                of contract law, regulatory frameworks, 
                and risk mitigation strategies.""",
    llm=llm,
    verbose=True
)


# Define a Writing Task
This block defines a writing task for the Senior Technical Writer agent. The task includes a description, the agent responsible, and the expected output.

In [3]:

writing_task = Task(
    description="""Write a well-structured, engaging,
                   and technically accurate article
                   on {topic}.""",
    
    agent=senior_technical_writer, 
    
    
    expected_output="""A polished, detailed, and easy-to-read
                       article on the given topic.""",
)


# Create a Crew and Execute the Task
This block creates a crew to manage the task and agents. It then kicks off the crew with the specified input and displays the response.

In [4]:
crew = Crew(
    agents=[senior_technical_writer],
    tasks=[writing_task],
    verbose=True
)

response = crew.kickoff(inputs={"topic":"AI Agents"})

# Display the response in Markdown format
# Markdown(response.raw)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8524423c-7c2d-42fd-8e88-bd3aa58a1458                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 288b178e-566a-4284-b494-6b0cd04175bd
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 288b178e-566a-4284-b494-6b0cd04175bd
       Status: Executing Task...
    └── 🤖 Agent: Senior Technical Writer
            Status: In Progress

# Agent: Senior Technical Writer
## Task: Write a well-structured, engaging,
                   and technically accurate article
                   on AI Agents.


🤖 Agent: Senior Technical Writer
    Status: In Progress
└── 🧠 Thinking...

...



# Agent: Senior Technical Writer
## Final Answer: 
**Title: Understanding AI Agents: An In-Depth Exploration**

**Introduction**

In the rapidly evolving landscape of artificial intelligence (AI), one concept that stands out for its transformative potential is that of AI agents. These autonomous systems are designed to perform a wide range of tasks, making decisions and learning from their interactions with the environment. This article aims to demystify AI agents, exploring their capabilities, types, application areas, and the challenges they pose. 

**What Are AI Agents?**

An AI agent is a software entity that perceives its environment through sensors and acts upon that environment using actuators. It operates autonomously, meaning it doesn't require human intervention to make decisions or perform tasks. The primary objective of an AI agent is to achieve specific goals by following a set of rules or learning from data.

**Key Characteristics of AI Agents**

1. **Autonomy:** AI age

🚀 Crew: crew
└── 📋 Task: 288b178e-566a-4284-b494-6b0cd04175bd
       Status: Executing Task...
    └── 🤖 Agent: Senior Technical Writer
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 288b178e-566a-4284-b494-6b0cd04175bd
       Assigned to: Senior Technical Writer
       Status: ✅ Completed
    └── 🤖 Agent: Senior Technical Writer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 288b178e-566a-4284-b494-6b0cd04175bd                                                                     │
│  Agent: Senior Technical Writer                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8524423c-7c2d-42fd-8e88-bd3aa58a1458                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Save the Response to a File
This block saves the raw response from the crew execution to a Markdown file for further use or documentation.

In [11]:
f = open("build/output.md", "w")
f.write(response.raw)
f.close()

# Define a file reader tool

In [12]:
from crewai_tools import FileReadTool

file_read_tool = FileReadTool()

# Define the Agent that will be responsible for reading and summarizing files

In [23]:
from crewai import Agent

summarizer_agent = Agent(
    role="Senior Document Summarizer",
    # goal="Summarize articles in exactly 20 words.",
    # backstory="You are an expert at creating concise summaries.",
    # llm=llm,
    goal="Extract and summarize key insights from provided files in 20 words or less.",
    backstory="""You are an expert in document analysis, skilled at extracting 
                 key details, summarizing content, and identifying critical 
                 insights from structured and unstructured text.""",
    llm=llm,
    tools=[file_read_tool],
    verbose=True
)

# Create a task that instructs the agent to read and analyze a file

In [24]:
from crewai import Task

summarizer_task = Task(
    description=(
        "Use the FileReadTool to read the contents of {file_path}"
        "and provide a summary in 20 words or less. "
        "Ensure the summary captures the key insights "
        "and main points from the document."
    ),
    agent=summarizer_agent,
    tools=[file_read_tool],
    expected_output="A concise 20-word summary of the key points from the file.",
)

# Assemble a Crew that executes the file-processing workflow

In [25]:
from crewai import Crew

summarizer_crew = Crew(
    agents=[summarizer_agent],
    tasks=[summarizer_task],
    verbose=True
)

# Run the crew with the file input
result = summarizer_crew.kickoff(inputs={"file_path": "build/output.md"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d813d2c0-8f2e-427b-9ecb-346152091caf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 32e830d8-a462-4ef6-83f1-2d8b2c6a1ee9
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 32e830d8-a462-4ef6-83f1-2d8b2c6a1ee9
       Status: Executing Task...
    └── 🤖 Agent: Senior Document Summarizer
            Status: In Progress

# Agent: Senior Document Summarizer
## Task: Use the FileReadTool to read the contents of build/output.mdand provide a summary in 20 words or less. Ensure the summary captures the key insights and main points from the document.


🤖 Agent: Senior Document Summarizer
    Status: In Progress

...



# Agent: Senior Document Summarizer
## Final Answer: 
AI agents autonomously learn, adapt, and interact, revolutionizing industries like healthcare and finance but pose complexity and ethical challenges.




🚀 Crew: crew
└── 📋 Task: 32e830d8-a462-4ef6-83f1-2d8b2c6a1ee9
       Status: Executing Task...
    └── 🤖 Agent: Senior Document Summarizer
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 32e830d8-a462-4ef6-83f1-2d8b2c6a1ee9
       Assigned to: Senior Document Summarizer
       Status: ✅ Completed
    └── 🤖 Agent: Senior Document Summarizer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 32e830d8-a462-4ef6-83f1-2d8b2c6a1ee9                                                                     │
│  Agent: Senior Document Summarizer                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d813d2c0-8f2e-427b-9ecb-346152091caf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [26]:
# Display the response in Markdown format
from IPython.display import Markdown
Markdown(result.raw)

AI agents autonomously learn, adapt, and interact, revolutionizing industries like healthcare and finance but pose complexity and ethical challenges.